In [4]:
from copy import deepcopy
import numpy as np
from random import shuffle

<h2><center> CSCI - UA 9472 - Artificial Intelligence </center></h2>

<h3><center> Assignment 3: Logical reasoning </center></h3>

<center>Given date: November 8 
</center>
<center><font color='red'>Due date: November 30 </font>
</center>
<center><b>Total: 40 pts </b>
</center>


<center>In this third assignment, we will implement a simple Logical agent by relying on the resolution algorithm of Propositional Logic.</center>

<img src="simpleVideoGameCave.jpeg" width="400" height="300"/>


### Introduction: logical propositions

The final objective will be to code our logical agent to succeed in a simple world similar to the Wumpus world discussed in the lectures. The final world we will consider is shown below.

<img src="MazeTotal.png" width="400" height="300"/>

Before designing the full agent, we will focus on a series of simplified environments (see below). In order to help you in your implementation, you are provided with the class 'Expr' and the associated function 'expr' which can be used to store and process logical propositions. The logical expressions are stored as objects consisting of an operator 'op' which can be of the types '&' (and), '|' (or) '==>' (implication) or '<=>' (double implication) as well as '~' (not). A logical expression such as 'A & B' can be stored as a string by means of the function expr() as expr('A & B') or Expr('&', 'A', 'B').

The function expr() takes operator precedence into account so that the two lines

In [5]:
'''source : AIMA'''

import collections

Number = (int, float, complex)
# Expression = (Expr, Number) # moved to lower part

def Symbol(name):
    """A Symbol is just an Expr with no args."""
    return Expr(name)


class Expr:
    """source: Artificial Intelligence: A Modern Approach
    A mathematical expression with an operator and 0 or more arguments.
    op is a str like '+' or 'sin'; args are Expressions.
    Expr('x') or Symbol('x') creates a symbol (a nullary Expr).
    Expr('-', x) creates a unary; Expr('+', x, 1) creates a binary."""
    
    def __init__(self, op, *args):
        self.op = str(op)
        self.args = args

    # Operator overloads
    def __neg__(self):
        return Expr('-', self)

    def __pos__(self):
        return Expr('+', self)

    def __invert__(self):
        return Expr('~', self)

    def __add__(self, rhs):
        return Expr('+', self, rhs)

    def __sub__(self, rhs):
        return Expr('-', self, rhs)

    def __mul__(self, rhs):
        return Expr('*', self, rhs)

    def __pow__(self, rhs):
        return Expr('**', self, rhs)

    def __mod__(self, rhs):
        return Expr('%', self, rhs)

    def __and__(self, rhs):
        return Expr('&', self, rhs)

    def __xor__(self, rhs):
        return Expr('^', self, rhs)

    def __rshift__(self, rhs):
        return Expr('>>', self, rhs)

    def __lshift__(self, rhs):
        return Expr('<<', self, rhs)

    def __truediv__(self, rhs):
        return Expr('/', self, rhs)

    def __floordiv__(self, rhs):
        return Expr('//', self, rhs)

    def __matmul__(self, rhs):
        return Expr('@', self, rhs)

    def __or__(self, rhs):
        """Allow both P | Q, and P |'==>'| Q."""
        if isinstance(rhs, Expression):
            return Expr('|', self, rhs)
        else:
            return PartialExpr(rhs, self)

    # Reverse operator overloads
    def __radd__(self, lhs):
        return Expr('+', lhs, self)

    def __rsub__(self, lhs):
        return Expr('-', lhs, self)

    def __rmul__(self, lhs):
        return Expr('*', lhs, self)

    def __rdiv__(self, lhs):
        return Expr('/', lhs, self)

    def __rpow__(self, lhs):
        return Expr('**', lhs, self)

    def __rmod__(self, lhs):
        return Expr('%', lhs, self)

    def __rand__(self, lhs):
        return Expr('&', lhs, self)

    def __rxor__(self, lhs):
        return Expr('^', lhs, self)

    def __ror__(self, lhs):
        return Expr('|', lhs, self)

    def __rrshift__(self, lhs):
        return Expr('>>', lhs, self)

    def __rlshift__(self, lhs):
        return Expr('<<', lhs, self)

    def __rtruediv__(self, lhs):
        return Expr('/', lhs, self)

    def __rfloordiv__(self, lhs):
        return Expr('//', lhs, self)

    def __rmatmul__(self, lhs):
        return Expr('@', lhs, self)

    def __call__(self, *args):
        """Call: if 'f' is a Symbol, then f(0) == Expr('f', 0)."""
        if self.args:
            raise ValueError('Can only do a call for a Symbol, not an Expr')
        else:
            return Expr(self.op, *args)

    # Equality and repr
    def __eq__(self, other):
        """x == y' evaluates to True or False; does not build an Expr."""
        return isinstance(other, Expr) and self.op == other.op and self.args == other.args

    def __lt__(self, other):
        return isinstance(other, Expr) and str(self) < str(other)

    def __hash__(self):
        return hash(self.op) ^ hash(self.args)

    def __repr__(self):
        op = self.op
        args = [str(arg) for arg in self.args]
        if op.isidentifier():  # f(x) or f(x, y)
            return '{}({})'.format(op, ', '.join(args)) if args else op
        elif len(args) == 1:  # -x or -(x + 1)
            return op + args[0]
        else:  # (x - y)
            opp = (' ' + op + ' ')
            return '(' + opp.join(args) + ')'


Expression = (Expr, Number)


def expr(x):
    """Shortcut to create an Expression. x is a str in which:
    - identifiers are automatically defined as Symbols.
    - ==> is treated as an infix |'==>'|, as are <== and <=>.
    If x is already an Expression, it is returned unchanged. Example:
    >>> expr('P & Q ==> Q')
    ((P & Q) ==> Q)
    """
    return eval(expr_handle_infix_ops(x), defaultkeydict(Symbol)) if isinstance(x, str) else x

def expr_handle_infix_ops(x):
    """Given a str, return a new str with ==> replaced by |'==>'|, etc.
    >>> expr_handle_infix_ops('P ==> Q')
    "P |'==>'| Q"
    """
    for op in infix_ops:
        x = x.replace(op, '|' + repr(op) + '|')
    return x

infix_ops = '==> <== <=>'.split()

class defaultkeydict(collections.defaultdict):
    """Like defaultdict, but the default_factory is a function of the key.
    >>> d = defaultkeydict(len); d['four']
    4
    """

    def __missing__(self, key):
        self[key] = result = self.default_factory(key)
        return result

    
class PartialExpr:
    """Given 'P |'==>'| Q, first form PartialExpr('==>', P), then combine with Q."""

    def __init__(self, op, lhs):
        self.op, self.lhs = op, lhs

    def __or__(self, rhs):
        return Expr(self.op, self.lhs, rhs)

    def __repr__(self):
        return "PartialExpr('{}', {})".format(self.op, self.lhs)



#### Question 1: to CNF (7pts)

Now that we can create a knowledge base, in order to implement the resolution algorithm that will ultimately enable our agent to leverage the information from the environment, we need our sentences to written in conjunctive normal form (CNF). That requires a number of steps which are recalled below:

- bicond elimination: $(\alpha \Leftrightarrow \beta) \equiv ((\alpha\Rightarrow \beta) \wedge (\beta \Rightarrow \alpha))$
- Implication elimination $\alpha \Rightarrow \beta \equiv \lnot \alpha \vee \beta$
- De Morgan's Law $\lnot (A\wedge B) = (\lnot A \vee \lnot B)$, $\lnot(\alpha \vee B) \equiv (\lnot A \wedge \lnot B)$
- Distributivity of $\vee$ over $\wedge$: $(\alpha \vee (\beta \wedge \gamma)) \equiv ((\alpha \vee \beta) \wedge (\alpha \vee \gamma))$

Relying on the function propositional_exp given above (to avoid having all the questions depend on question 1, we will now rely on this implementation from AIMA), complete the function below which should return the CNF of the logical propostion $p$.  

The functions I wrote to achieve CNF does not return anything. It computes everything in-place. 

In [6]:
def is_op_free(p,op):
    return not op in p.__repr__()
    
    
def remove_bicond(p):
    
    if is_op_free(p,'<=>'):
        return 

    op = p.op
    if op == '<=>':
        assert(len(p.args)==2)
        l,r = p.args
        remove_bicond(l)
        remove_bicond(r)
        p.op = '&'
        p.args = (Expr('==>', l, r), Expr('==>', r, l))
        return
    else:
        for i in p.args:
            remove_bicond(i)
        return


In [7]:
def remove_impl(p):
    assert(is_op_free(p,'<=>'))
    
    if is_op_free(p, '==>'):
        return 
        
    op = p.op
    
    if op == '==>':
        assert(len(p.args) == 2)
        l, r = p.args
        remove_impl(l)
        remove_impl(r)        
        p.op = '|'
        p.args = (Expr('~',l), Symbol(r))
        return
        
    else:
        for i in p.args: 
            remove_impl(i)
        return
    
def is_atomic(p):
    if p.args == ():
        return True
    if p.op == '~':
        if len(p.args) == 1:
            a, = p.args
            if a.args == ():
                return True
    return False

def move_neg(p):

    # assert(is_op_free(p,'==>') and is_op_free(p,'<=>'))
    
    if is_atomic(p):
        return 

    if p.op == '~':
        assert(len(p.args) == 1)
        a, = p.args
        assert(a.op in ['~','|','&'])
        
        if a.op == '~':
            # double negation cancels
            # so I need to make this p as the same as the a.args[0]
            try:
                p.op = a.args[0].op
                p.args = a.args[0].args
            except AttributeError:
                p.op = a.args[0]
            return
        
        else:
            if a.op == '|':
                p.op = '&'
            elif a.op == '&':
                p.op = '|'
            else:
                assert(False)
                
            p.args = tuple([Expr('~', i) for i in a.args])
            
    for i in p.args: 
        move_neg(i)


it could be tricky to apply the distribution law. So I better document it here. 

The general rules are:
- if p is atomic, then do nothing
- if p is conjunction of CNF, then just lift each args of the CNF
- if p is disjunction of 2 CNF, then apply the distribution law as following

For two conjunctive normal forms $CNF_1 = \wedge_i (CNF_{1,i}), CNF_2 = \wedge_j (CNF_{2,j})$, where each of $CNF_{1,i},CNF_{2,j}$ are just disjunction of atomic symbols. I can have the following calculation 

$$
CNF_1 \vee CNF_2 \\
= (\wedge_i CNF_{1,i} ) \vee CNF_2 \\
= \wedge_i (CNF_{1,i} \vee CNF_2) \\
= \wedge_i (CNF_{1,i} \vee (\wedge_j CNF_{2,j} )) \\
= \wedge_i (\wedge_j (CNF_{1,i} \vee CNF_{2,j}) ) \\
= \wedge_{i,j} (CNF_{1,i} \vee CNF_{2,j})
$$

More generally, disjunction of $n$ CNFs can be computed as 

$$
CNF_1 \vee CNF_2 \vee \cdots \vee CNF_n \\
= \wedge_{i_1,i_2, i_n} (CNF_{1,i_1} \vee CNF_{2,i_2} \vee \cdots \vee CNF_{n,i_n}) \\
= \wedge_{i_1,i_2, i_n} ( \vee_{t=1}^n CNF_{t,i_t}) \\
$$

- Also, one should remove repeated atomic symbols whenever possible. 


In [8]:
def is_cnf_components(p):
    
    if is_atomic(p): return True
    
    if p.op != '|' : return False
    
    return all([is_atomic(a) for a in p.args])
    
    
def is_cnf(p):
    
    if is_cnf_components(p):
        return True
    
    if p.op != '&': return False
    
    return all([is_cnf_components(a) for a in p.args])
    
def get_cnf_components(p):
    assert(is_cnf(p))
    if is_cnf_components(p):
        return (p,)
    else:
        return p.args

def get_component_symbols(p):
    assert(is_cnf_components(p))
    if is_atomic(p):
        return [p]
    else:
        return list(p.args)

In [9]:
def move_or(p):
    if is_cnf(p):
        return
    
    if p.op == '&':
        # conjunction of CNF.
        new_components = []
        for a in p.args:
            move_or(a)
            new_components.extend(get_cnf_components(a))
        
        p.args = tuple(new_components)
        # ✔
    
    if p.op == '|':
        # disjunction of CNFs,
        # this can be tricky, be careful ❗ 
        dis_components = []
        for a in p.args:
            move_or(a)
            dis_components.append(get_cnf_components(a))
            
        # I will use the divide and conquer strategies here. In an aux function as following. 
        p.args = tuple(merge_disjunct_cnf_comp(dis_components))
        p.op = '&'
        

def merge_disjunct_cnf_comp(dis_components):
    '''
    inputs: the components of 2 cnf
    outputs: the components of 1 cnf
    '''
    n = len(dis_components)
    if n == 0:
        return None
        
    if n == 1:
        return dis_components[0]
    
    if n == 2:
        dis0 = dis_components[0]
        dis1 = dis_components[1]
        ret  = [disjunct_2_cnf_components(x,y) for x in dis0 for y in dis1]
        # 
        return ret
        
    if n > 2:
        mid = n//2
        first = merge_disjunct_cnf_comp(dis_components[:mid])
        second = merge_disjunct_cnf_comp(dis_components[mid:])
        return merge_disjunct_cnf_comp([first,second])


def disjunct_2_cnf_components(x,y):
    assert(is_cnf_components(x))
    assert(is_cnf_components(y))
    all_terms = get_component_symbols(x) + get_component_symbols(y)
    pos = set() # I am using set to remove duplicates. 
    neg = set()
    
    for s in all_terms:
        if s.args == ():
            pos.add(s.op)
        else:
            assert(is_atomic(s))
            neg.add(s.args[0].op)
    
    ret = [expr(s) for s in pos]
    ret.extend([expr('~'+s) for s in neg])
        
    
    # ret = [expr(s) for s in pos]
    # ret.extend([expr('~'+s) for s in neg])
        
    p = Expr('|')
    p.args = tuple(ret)
    
    return p
    
    # ❗ I choose not to resolve `a or not a`. Because I don't think we assume law of excluding the middle. 

In [10]:
def to_CNF(p):
    
    '''function should return the CNF of proposition p'''
    
    remove_bicond(p)
    remove_impl(p)
    move_neg(p)
    move_or(p)
    return p


In [11]:
t = expr("~(a<=>b)")
t = to_CNF(t)
print(t)

((a | b) & (a | ~a) & (b | ~b) & (~b | ~a))


(b or not b) and (b or c) 
(b or c)

(a or not a) and (b or c)



#### Question 2: The resolution rule (7pts)

Now that you have a function that can turn any logical sentence to a CNF, we will code the resolution rule. For any two propositions $p_1$ and $p_2$ written in conjunctive normal form, write a function Resolution that returns empty if the resolution rule applied to the two sentences cannot produce any new sentence and that returns the set of all propositions $p_i$ following from the resolution of $p_1$ and $p_2$ otherwise.  

Study the disjuncts of both $p_1$ and $p_2$. For each of the disjunct in $p_1$ try to find in $p_2$ the negation of that disjunct. If this negation appears, returns the proposition resulting from combining $p_1$ and $p_2$ with a disjunction. 

In [12]:
def resolution_rule(p1, p2):
    
    '''applies the resolution rule on two propositions p1 and p2'''
    
    assert(is_cnf_components(p1) and is_cnf_components(p2))
    
    p1s = get_component_symbols(p1)
    p2s = get_component_symbols(p2)
    
    p1pos = set([i for i in p1s if i.op != '~'])
    p1neg = set([Symbol(i.args[0]) for i in p1s if i.op == '~'])
    
    p2pos = set([i for i in p2s if i.op != '~'])
    p2neg = set([Symbol(i.args[0]) for i in p2s if i.op == '~'])
    
    if len(p1pos.intersection(p2neg)) == 0 and len(p2pos.intersection(p1neg)) == 0:
        return 'nothing to resolve'
    
    total_pos = (p1pos - p2neg) | (p2pos - p1neg) 
    total_neg = (p1neg - p1pos) | (p2neg - p1pos)
    
    
    args = list(total_pos) + [Expr('~',i) for i in total_neg]
    
    if len(args) == 1:
        return Symbol(*args)
    if len(args) == 0:
        return expr('()')
    
    return Expr('|', *args)


#### Question 3: The resolution algorithm (6pts)

Now that we have a resolution function we can embed it into a resolution algorithm. Complete the function Resolution below which implements the resolution algorithm. The algorithm takes as input a knowledge base written in conjunctive normal form, and a proposition $\alpha$ and should return true or false (as stored in the variable 'is_entailed') depending on whether $\alpha$ is entailed by the knowledge base KB.

In [13]:
class KnowledgeBase:
    def __init__(self, *props):
        
        self.records = {}
        
        for prop in props:
            prop = to_CNF(prop)
            for cnf_comp in get_cnf_components(prop):
                self.records[cnf_comp.__repr__()] = cnf_comp
    
    def resolution(self, alpha):
        
        new_records = {i.__repr__():i for i in get_cnf_components(to_CNF(Expr('~', alpha)))}
        all_records = deepcopy(self.records)
        all_records.update(new_records)
        
        resolved = []

        while new_records:
            new_records = dict()
            for i,p in all_records.items():
                for j,q in all_records.items():
                    if (i,j) in resolved:
                        continue
                        # I have already resolved it, so just continue. 
                    res = resolution_rule(p, q)
                    resolved.append((i, j))
                    
                    if res == 'nothing to resolve':
                        continue
                    
                    if res == expr('()'):
                        return True
                    
                    if res.__repr__() not in all_records.keys():
                        new_records[res.__repr__()] = res
                        
            all_records.update(new_records)
            
        return False
        
    def learn(self, prop):
        self.records.update(
            {i.__repr__(): i for i in get_cnf_components(to_CNF(prop))})
            
    def check(self,alpha):
        if KB.resolution(alpha):
            self.learn(alpha)
            return True
        else:
            return False
    


#### Question 4 (8pts): A first logical agent

Given our resolution algorithm, we will finally be able to design our first logical agent. As a first step, we consider a simple agent located on the bottom left cell of a 5 by 5 grid world. The world contains a single threat represented by a ghost which occupies a single cell and emits a loud noise ('OOooo') audible in the immediately adjacent cells. 

To implement the agent, we will use the following approach:

Each cell will be represented by its (x,y) coordinate ((0,0) being the bottom leftmost cell). On top of this we will consider the following symbols 

- $D_{(x, y)}$ (which you can store as the string 'Dxy' or 'D_xy' as you want) indicating whether there is an **exit** on the cell or not (when the agent reach the exit, the simulation ends), 

- $G_{(x, y)}$ which encodes whether the **ghost** is located on cell $(x, y)$ or not

- $O_{(x, y)}$ which encodes whether a "Ooooo" is **heard** on the cell $(x, y)$

Using those three symbols, the state of the world can be defined with a total of 3*25 symbols. 

In a while loop running until the door is found, code a simple logical agent that moves at random in the non-threatening cells until it finds the escape cell. Also consider the following:

- The agent should keep track of a knowledge base KB (list of logically true propositions together with a dictionnary storing the symbols that are known to be true) including all the sentences that the agent can generate based on the information it collected at the previous steps. 


- You might want to have a simple function, which given a symbol returns the adjacent symbols (symbols corresponding to spatially adjacent cells). you can store those as strings


- The agent should be equipped with the resolution algorithm that you coded above. Before each new move, the next cell should be determined at random (from the set of all non visited cells) and the agent should use the resolution algorithm to determine whether the ghost is on the cell or not. If there is no indication that the ghost is located on the cell, the agent should make the move. 

<img src="MazeGhostb.png" width="400" height="300"/>

In [14]:
# aux functions
def adjacent_cells(pos):
    x,y = pos
    adj = []
    if x > 0:
        adj.append((x-1,y))
    if x < 4:
        adj.append((x+1,y))
    if y > 0:
        adj.append((x,y-1))
    if y < 4:
        adj.append((x,y+1))
    shuffle(adj)
    # this steps gaurantee that the next step is at random
    return adj
def pos2str(pos):
    return '_' + str(pos[0]) + '_' + str(pos[1])


In [15]:
# env set up
worlds = np.zeros((5,5))
worlds[2,1] = -1 # marking the ghost
worlds[-1,-1] = 1 # marking the exit
for i in adjacent_cells((2,1)):
    worlds[i] = -2 # marking the Ooo
current_pos = (0,0)
print(worlds[::-1])

[[ 0.  0.  0.  0.  1.]
 [ 0. -2.  0.  0.  0.]
 [-2. -1. -2.  0.  0.]
 [ 0. -2.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]]


In [16]:
# building the rules of this game: if there is a G_x_y, then for adjcent cells, O_adj
rules = []
for x in range(5):
    for y in range(5):
        pos = (x,y)
        for adj in adjacent_cells(pos):
            rules.append(Expr('==>', expr('G'+pos2str(pos)), expr('O'+pos2str(adj))))
            
KB = KnowledgeBase(*rules)

In [19]:
exitNotFound = True

current_pos = (0,0)
path = []
x_max = 0
y_max = 0

count = 0


while exitNotFound:

    '''Agent should explore the world at random, probing the 
    knowledge base for any potential threat. if the KB does not 
    indicate any specific threat, the agent should move in one of 
    the adacent (cleared) cell. The simulation ends when the agent 
    reaches the exit door'''
    
    # evaluating current and learning
    path.append(current_pos)
    print(current_pos)
    count += 1
    if worlds[current_pos] == -1:
        print('game over, I have met the ghost. ')
        assert(False)
    elif worlds[current_pos] == -2:
        KB.learn(expr('O' + pos2str(current_pos)))
        KB.learn(expr('~G' + pos2str(current_pos)))
    elif worlds[current_pos] == 1:
        exitNotFound = False
        print("game won, reached the exit")
        break
    else: # worlds[current_pos] == 0
        KB.learn(expr('~O' + pos2str(current_pos)))
        KB.learn(expr('~G' + pos2str(current_pos)))
    
    # next step randomly
    for next_pos in adjacent_cells(current_pos):
        # check if its safe
        if KB.check(expr('~G' + pos2str(next_pos))):
            current_pos = next_pos
            break

    if count > 1000:
        break



(0, 0)
(1, 0)
(0, 0)
(0, 1)
(0, 2)
(0, 1)
(0, 0)
(0, 1)
(1, 1)
(0, 1)
(1, 1)
(0, 1)
(0, 2)
(1, 2)
(2, 2)
(1, 2)
(0, 2)
(0, 1)
(0, 0)
(1, 0)
(0, 0)
(0, 1)
(1, 1)
(0, 1)
(1, 1)
(1, 2)
(0, 2)
(0, 1)
(1, 1)
(0, 1)
(1, 1)
(1, 2)
(1, 1)
(0, 1)
(0, 2)
(0, 3)
(0, 4)
(1, 4)
(0, 4)
(1, 4)
(1, 3)
(0, 3)
(0, 2)
(1, 2)
(2, 2)
(2, 3)
(3, 3)
(2, 3)
(2, 4)
(1, 4)
(0, 4)
(1, 4)
(0, 4)
(0, 3)
(1, 3)
(2, 3)
(1, 3)
(0, 3)
(0, 4)
(0, 3)
(1, 3)
(0, 3)
(1, 3)
(1, 4)
(1, 3)
(2, 3)
(2, 2)
(3, 2)
(2, 2)
(2, 3)
(1, 3)
(1, 4)
(2, 4)
(3, 4)
(4, 4)
game won, reached the exit


In [23]:
len(path), len(set(path))

(75, 17)

It took about 15-20 minutes for the agent to finish this game, so it is very **inefficient**. 

#### Question 5: Getting used to danger.. (6pts)

Now that our agent knows how to handle a ghost, we will increase the level of risk by including spike traps in the environment. 

- Our agent has an edge: evolution has endowed it with a sort of additional ($6^{th}$) sense so that it can feel something 'bad' is about to happen when it is in a cell adjacent to a trap. We represent this ability with the eye combined with the exclamation mark. 


- Since, as we all know, ghosts are purely imaginary entities, the $6th$ sense only works for the spike traps.  


- The ghost can still be located by means of the noise it generates which can be heard on all adjacent cells.

<img src="MazeTrapb.png" width="400" height="300"/>

Starting from the agent you designed in the previous questions, improve this agent so that it takes into account the spike traps. You should now have a knowledge base defined on a total of 25*5 symbols describing whether each cell contains a ghost, a 'OOoo' noise, a spike trap, activated the agent's'6th sense', or contains the exit door. The search ends when the agent reaches the door.   

In [26]:
worlds[(1,2),(3,4)] = -3

In [40]:
ghost_rules = []
for x in range(5):
    for y in range(5):
        pos = (x, y)
        for adj in adjacent_cells(pos):
            ghost_rules.append(
                Expr('==>', expr('G'+pos2str(pos)), expr('O'+pos2str(adj))))

spike_rules = []
for x in range(5):
    for y in range(5):
        pos = (x, y)
        for adj in adjacent_cells(pos):
            spike_rules.append(
                Expr('==>', expr('Spike'+pos2str(pos)), expr('Six_sense'+pos2str(adj))))

rules = ghost_rules + spike_rules

KB = KnowledgeBase(*rules)


In [47]:
# env set up

ghost_flag = 1
Ooooo_flag = 2
spike_flag = 4
six_sense_flag = 8
exit_flag = 16

worlds = np.zeros((5,5),dtype=np.ubyte)
worlds[2, 1] |= ghost_flag  # marking the ghost
worlds[(1,2),(4,3)] |= spike_flag # marking the spikes
worlds[-1,-1] |= exit_flag # marking the exit
for i in adjacent_cells((2,1)):
    worlds[i] |= Ooooo_flag # marking the Ooo
for i in adjacent_cells((1,2)) + adjacent_cells((3,4)):
    worlds[i] |= six_sense_flag # marking the 6th sense
current_pos = (0,0)
print(worlds[::-1])


[[ 0  0  0  0 24]
 [ 0  2  0  8  4]
 [ 2  1 10  0  8]
 [ 0 10  4  8  0]
 [ 0  0  8  0  0]]


In [49]:
exit_door = False 
curr_pos = (0,0)
max_Iter = 1000
path2 = []

while (exit_door != True) and max_Iter > 0:
    
    '''Complete the loop with the simulation of the ghost 
    + spike trap logical agent. The simulation should start 
    from the bottom leftmost cell'''
    
    max_Iter += -1
    path2.append(curr_pos)
    
    if worlds[curr_pos] & ghost_flag:
        print("killed by the ghost, game over")
        assert(False)
    else:
        KB.learn(expr("~G" + pos2str(curr_pos)))
        
    if worlds[curr_pos] & Ooooo_flag:
        KB.learn(expr("O"+pos2str(curr_pos)))
    else:
        KB.learn(expr("~O"+pos2str(curr_pos)))
        
    if worlds[curr_pos] & spike_flag:
        print("killed by the spike, game over")
        assert(False)
    else:
        KB.learn(expr("~Spike"+pos2str(curr_pos)))
        
    if worlds[curr_pos] & six_sense_flag:
        KB.learn(expr("Six_sense"+pos2str(curr_pos)))
    else:
        KB.learn(expr("~Six_sense"+pos2str(curr_pos)))
    if worlds[curr_pos] & exit_flag:
        exit_door == True
        continue
    
    for next_pos in adjacent_cells(curr_pos):
        if KB.check(expr("~(Spike" + pos2str(next_pos) + ")|(G" + pos2str(next_pos) + ")")):
            curr_pos = next_pos
            break

#### Bonus: For where your treasure is..  (6pts)

We finally consider the whole environment. This environment is composed of all the elements from the previous questions but it now also includes a treasure chest. The final objective this time is to find the chest first and then reach the exit. Although some of the previous symbols are omitted for clarity, the ghost can always be located by means of the sound it produces, the agent can still trust its $6^{\text{th}}$ sense regarding the spike trap and the treasure chest can be perceived in adjacent cells, by means of the shine it produces.

When the knowledge base does not indicate any threat, the agent should move at random in one of the adjacent cells. 

The world now contains a total of 25*7 symbols. 


<img src="fullWorldChest.png" width="400" height="300"/>

In [ ]:
ghost_rules = []
for x in range(5):
    for y in range(5):
        pos = (x, y)
        for adj in adjacent_cells(pos):
            ghost_rules.append(
                Expr('==>', expr('G'+pos2str(pos)), expr('O'+pos2str(adj))))

spike_rules = []
for x in range(5):
    for y in range(5):
        pos = (x, y)
        for adj in adjacent_cells(pos):
            spike_rules.append(
                Expr('==>', expr('Spike'+pos2str(pos)), expr('Six_sense'+pos2str(adj))))
                
treasure_rules = []
for x in range(5):
    for y in range(5):
        pos = (x, y)
        for adj in adjacent_cells(pos):
            spike_rules.append(
                Expr('==>', expr('Treasure'+pos2str(pos)), expr('Shine'+pos2str(adj))))

rules = ghost_rules + spike_rules + treasure_rules

KB = KnowledgeBase(*rules)

In [1]:
# env set up

ghost_flag = 1
Ooooo_flag = 2
spike_flag = 4
six_sense_flag = 8
treasure_flag = 32
shine_flag = 64
exit_flag = 128

ghosts = [(1,2)]
treasures = [(3,4)]
spikes = [(2,1),(0,2),(4,3)]

worlds = np.zeros((5,5),dtype=np.ubyte)

for g in ghosts:
    world[g] |= ghost_flag
    for o in adjacent_cells(g):
        world[o] |= Ooooo_flag
for g in spikes:
    world[g] |= spike_flag
    for o in adjacent_cells(g):
        world[o] |= six_sense_flag
for g in treasures:
    world[g] |= treasure_flag
    for o in adjacent_cells(g):
        world[o] |= shine_flag

world[(-1,-1)] |= exit_flag

current_pos = (0,0)
print(worlds[::-1])


NameError: name 'np' is not defined

In [ ]:
exit_found = False
treasure_found = False
 
curr_pos = (0,0)
max_Iter = 1000

path3 = []

while ((exit_found != True) or (treasure_found != True)) and max_Iter > 0:
    
    max_Iter += -1
    path3.append(curr_pos)
    
    if worlds[curr_pos] & ghost_flag:
        print("killed by the ghost, game over")
        assert(False)
    else:
        KB.learn(expr("~G" + pos2str(curr_pos)))
        
    if worlds[curr_pos] & Ooooo_flag:
        KB.learn(expr("O"+pos2str(curr_pos)))
    else:
        KB.learn(expr("~O"+pos2str(curr_pos)))
        
    if worlds[curr_pos] & spike_flag:
        print("killed by the spike, game over")
        assert(False)
    else:
        KB.learn(expr("~Spike"+pos2str(curr_pos)))
        
    if worlds[curr_pos] & six_sense_flag:
        KB.learn(expr("Six_sense"+pos2str(curr_pos)))
    else:
        KB.learn(expr("~Six_sense"+pos2str(curr_pos)))
    
    if worlds[curr_pos] & treasure_flag:
        KB.learn(expr("Treasure"+pos2str(curr_pos)))
        treasure_found = True
    else:
        KB.learn(expr("~Treasure"+pos2str(curr_pos)))
    
    if worlds[curr_pos] & shine_flag:
        KB.learn(expr("Shine"+pos2str(curr_pos)))
    else:
        KB.learn(expr("~Shine"+pos2str(curr_pos)))
    
    if worlds[curr_pos] & exit_flag:
        exit_found == True
    
    for next_pos in adjacent_cells(curr_pos):
        if KB.check(expr("~(Spike" + pos2str(next_pos) + ")|(G" + pos2str(next_pos) + ")")):
            curr_pos = next_pos
            break
